## Lecture 6  Part 1
## Introduction to Numba
### Feb. 28, 2024

## Reminder

**Before next week**: send me email with 5 members of your project group!

**Next week**: project proposal presentations (likely remote only), 5 min for each group (+ a few min for questions)

Part of this lecture is based on the material: 

see [https://nyu-cds.github.io/python-numba/](https://nyu-cds.github.io/python-numba/)

You will need the Numba package for this lecture (anaconda already installed it): 

[https://anaconda.org/numba/numba](https://anaconda.org/numba/numba)

### Introduction to Numba

----
- Numba provides the ability to speed up applications with high performance functions written directly in Python, rather than using language extensions such as Cython.

- Numba allows the compilation of selected portions of pure Python code to native code, and generates optimized machine code.

- With a few simple annotations, array-oriented and math-heavy Python code can be **just-in-time (JIT)** optimized to achieve performance similar to C and C++, without having to switch languages or Python interpreters.

- **just-in-time (JIT)**: is complition of code during execution of a program (at run-time) rather than before execution.

Numba’s main features are:

- On-the-fly code generation (at import time or runtime, at the user’s preference):(Modified code while it is running, without stopping or restarting).  
- Native code generation for the CPU (default) and GPU hardware  
- Integration with the Python scientific software stack (thanks to NumPy)



**Numba**: 
- is a just-in-time compiler for Python that works best on code that uses NumPy arrays and functions, and loops. 
- The most common way to use Numba is through its collection of decorators that can be applied to your functions to instruct Numba to compile them. When a call is made to a Numba decorated function it is compiled to machine code “just-in-time” for execution and all or part of your code can subsequently run at native machine code speed!



- Numba’s central feature/most fundamental is the **numba.jit()** decorator (take a moment to recap function decoration we learned before), which marks a function for optimization by Numba’s JIT compiler.

- If your code is numerically orientated (does a lot of math), uses NumPy a lot and/or has a lot of loops, then Numba is often a good choice. (It won’t work very well, if at all, if you code uses Pandas).

Lets start with a simple example:

In [3]:
#First we’ll create an array of sorted values and randomly shuffle them.
import numpy as np

# an array of 1000 floats 0.0 to 9.99
original_array = np.arange(0.0, 10.0, 0.01, dtype='float')
#print(original_array)

In [4]:
# randomly shuffle

shuffled_array = original_array.copy()
np.random.shuffle(shuffled_array)
sorted_array = shuffled_array.copy()

Let’s see Numba in action. The following is a Python implementation of bubblesort for NumPy arrays.

In [4]:
# bubblesort as pure python code

def bubblesort(X):4
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

- Now let’s see time the execution. 

- Note: we need to copy the array so we sort a random array each time as sorting an already sorted array is faster and so would distort our timing.

In [5]:
# Pure Python implementation is
%timeit -n 10 sorted_array[:] = shuffled_array[:]; bubblesort(sorted_array)

print("length:", len(original_array))
print("original: ", original_array[:10])
print("shuffled: ", shuffled_array[:10])
print("sorted:   ", sorted_array[:10])

158 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
length: 1000
original:  [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]
shuffled:  [9.64 0.03 8.4  5.37 8.63 3.48 1.71 0.97 6.77 5.57]
sorted:    [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]


- Now we simply use the **@jit** decorator (or **@njit** if there is Deprecation Warnings) and let Numba decide when and how to optimize, so we simply add the decorator to the function:

In [6]:
from numba import jit

#@jit 
@jit(nopython=True)  # Set "nopython" mode for best performance, equivalent to @njit
def bubblesort_numba(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp+1

In [7]:
%time sorted_array[:] = shuffled_array[:]; bubblesort_numba(sorted_array)

CPU times: total: 188 ms
Wall time: 950 ms


In [8]:
%timeit -n 10 sorted_array[:] = shuffled_array[:]; bubblesort_numba(sorted_array)

636 µs ± 66.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit -n 1000 sorted_array[:] = shuffled_array[:]; np.sort(sorted_array)

27 µs ± 5.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Function Signature

- It is also possible to specify the signature of the Numba function. 

- A function signature describes **the types of the arguments** and **the return type of the function**. 

- This can produce **slightly** faster code as the compiler does not need to infer the types. 

- However the function is no longer able to accept other types. The specified types within @jit called the function _signature_.

In [10]:
from numba import njit, int32, float64

@njit(float64(int32, int32))
def f(x, y):
    # A somewhat trivial example
    return (x + y) / 3.14

- In this example, float64(int32, int32) is the function’s signature specifying a function that takes two 32-bit integer arguments and returns a double precision float. Numba provides a **shorthand notation**, so the same signature can be specified as f8(i4, i4).

See shorthand notation:
https://numba.pydata.org/numba-doc/0.31.0/reference/types.html#numbers

- Specialization can make the functions less flexible: The specialization will be compiled by the @jit decorator, and no other specialization will be allowed. This is useful if you want fine-grained control over types chosen by the compiler (for example, to use single-precision floats).

- If you omit the return type, e.g. by writing (int32, int32) instead of float64(int32, int32), Numba will try to infer it for you. Function signatures can also be strings, and you can pass several of them as a list; see the numba.jit() documentation for more details.


In [11]:
f(2, 2)

1.2738853503184713

- Array signatures are specified by subscripting a base type according to the number of dimensions. For example a 1-dimension single-precision array would be written float32[:], or a 3-dimension array of the same underlying type would be f4[:,:,:] (using the shorthand notation).

In [12]:
from numba import njit, float64

@njit("void(float64[:])")
def bubblesort_numba_argtypes(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [13]:
%timeit -n 10 sorted_array[:] = shuffled_array[:]; bubblesort_numba_argtypes(sorted_array)

988 µs ± 106 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Signature specifications**

Explicit @jit signatures can use a number of types. Here are some common ones:

- _void_ is the return type of functions returning nothing (which actually return None when called from Python)

- _intp_ and _uintp_ are pointer-sized integers (signed and unsigned, respectively)

- _intc_ and _uintc_ are equivalent to C int and unsigned int integer types

- _int8_, _uint8_, _int16_, _uint16_, _int32_, _uint32_, _int64_, _uint64_ are fixed-width integers of the corresponding bit width (signed and unsigned)

- _float32_ and _float64_ are single- and double-precision floating-point numbers, respectively

- _complex64_ and _complex128_ are single- and double-precision complex numbers, respectively

- array types can be specified by indexing any numeric type, e.g. _float32[:]_ for a one-dimensional single-precision array or _int8[:,:]_ for a two-dimensional array of 8-bit integers.

---
### Compilation Modes
Numba has two compilation modes: 
- nopython mode 
- object mode

**nopython mode**: 

- Numba compiler generates code that does not access Python C API.(The behaviour of the nopython compilation mode is to essentially compile the decorated function so that it will run entirely without the involvement of the Python interpreter)
- This mode produces the highest performance code, but requires that the native types of all values in the function can be inferred.  (This is the recommended and best-practice way to use the Numba jit decorator as it leads to the best performance.)


**object mode**:

- Numba compiler generates code that handles all values as Python objects and uses the Python C API to perform all operations on those objects.(In this mode Numba will identify loops that it can compile and compile those into functions that run in machine code, and it will run the rest of the code in the interpreter) 
- Code compiled in object mode will often run no faster than Python interpreted code. This mode is used when the type of some variables can not be inferred.


A typical approach is to force the **nopython** mode, triggering an error message when the mode is not possible.

- **nopython** mode is so useful that newer versions of numba provides a **njit** decorator.

In [6]:
from numba import njit, jit, float64

# @jit("void(float64[:])", nopython=True)
@njit("void(float64[:])")
def bubblesort_nopython_flag(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [15]:
%timeit -n 10 sorted_array[:] = shuffled_array[:]; bubblesort_nopython_flag(sorted_array)

953 µs ± 128 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Notice that this code compiles cleanly. However, if we introduce an object whose type cannot be inferred an error message shows up.

In [7]:
from decimal import Decimal

@jit("void(float64[:])", nopython=True)
def bubblesort_nopython_flag(X):
    N = len(X)
    val = Decimal(100)  # just to force an error
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

Now when we try to compile this code, Numba complains that Decimal is an untyped name. Without the nopython mode, this code would have compiled, but would have run much more slowly.

In [20]:
%timeit -n 10 sorted_array[:] = shuffled_array[:]; bubblesort_nopython_flag(sorted_array)

1.02 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Calling other functions
Numba functions can call other Numba functions. Both functions must have the **@jit** decorator, otherwise the code will be much slower.

In [80]:
import numpy as np

original = np.arange(0.0, 10.0, 0.01, dtype='float')
shuffled = original.copy()
np.random.shuffle(shuffled)

sorted = shuffled.copy()

In [82]:
from numba import jit, float64

@jit("void(float64[:])", nopython=True)
def bubblesort_ff(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp
               
@jit("void(float64[:])",nopython=True)
def do_sort(sorted):
    bubblesort_ff(sorted)
    

In [83]:
%timeit -n 10 sorted[:]=shuffled[:]; do_sort(sorted)

894 µs ± 112 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


---
###  NumPy Universal Functions ([ufunc](https://docs.scipy.org/doc/numpy-1.10.0/reference/ufuncs.html#universal-functions-ufunc))


---
- Examples of Numpy ufunc include add(), multiply(), and sin().  

- These functions **operate on ndarrays** in an **element-by-element** fashion, supporting array broadcasting, type casting, and several other standard features.

- A ufunc is a “vectorized” wrapper for a function that takes a fixed number of scalar inputs and produces a fixed number of scalar outputs.

- Many of NumPy’s builtin operators are ufuncs.

---
- Numba’s **@vectorize** decorator allows Python functions taking scalar input arguments to be used as **NumPy ufuncs**.   


- (Creating a traditional NumPy ufunc is not the most straightforward process and involves writing some C code. Numba makes this easy.)  


- Using the @vectorize decorator, Numba can compile a pure Python function into a ufunc that operates over NumPy arrays as fast as traditional ufuncs written in C.  

The @vectorize decorator has two modes of operation:

- **Eager**, or decoration-time, compilation. If you pass one or more type signatures to the decorator, you will be building a Numpy ufunc. We’re just going to consider eager compilation here.
- **Lazy**, or call-time, compilation. When not given any signatures, the decorator will give you a Numba dynamic universal function (DUFunc) that dynamically compiles a new kernel when called with a previously unsupported input type.  


**Using @vectorize, you write your function as operating over input scalars, rather than arrays. Numba will generate the surrounding loop (or kernel) allowing efficient iteration over the actual inputs.**   


In [84]:
import numpy as np
from numba import vectorize, int64

@vectorize([int64(int64, int64)])
def vec_add_vectorize(x, y):
    return x + y


In [85]:
a = np.arange(6, dtype=np.int64)
b = np.linspace(0, 10, 6, dtype=np.int64)

print("a    : ", a)
print("b    : ", b)

print("-" * 80)
print("a + a: ", vec_add_vectorize(a, a))
print("b + b: ", vec_add_vectorize(b, b))

a    :  [0 1 2 3 4 5]
b    :  [ 0  2  4  6  8 10]
--------------------------------------------------------------------------------
a + a:  [ 0  2  4  6  8 10]
b + b:  [ 0  4  8 12 16 20]


In [86]:
@jit("int64[:](int64[:], int64[:])")
def vec_add_jit(x, y):
    return x + y

C:\Users\hagha\AppData\Local\Temp\ipykernel_29768\3286286215.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit("int64[:](int64[:], int64[:])")


In [87]:
print(vec_add_jit(a, a))
print(vec_add_jit(b, b))

[ 0  2  4  6  8 10]
[ 0  4  8 12 16 20]


The difference between the **@vectorize** and **@jit** is that "vectorize" is creating a new function with the associated efficient for-loop, while "@jit" is using the Numpy function.

[http://numba.pydata.org/numba-doc/latest/user/vectorize.html](http://numba.pydata.org/numba-doc/latest/user/vectorize.html)